In [1]:
import os
import pylab
import cv2 as cv
import pickle
import numpy as np
import tensorflow as tf
from IPython.display import Image
from keras import Model
from keras import backend as K
from keras.applications import vgg16
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array

Using TensorFlow backend.


In [2]:
class Shingling:
        
    def generate_shingles(self, folder, width, height, method, layer=None):
        if method == "Sobel":
            return self.__generate_sobel_shingles(folder, width, height)
        elif method == "CNN" and layer != None:
            return self.__generate_cnn_shingles(folder, width, height, layer)
        
    ######################################## SOBEL ########################################
    
    # Generates sobel features for each image in given folder
    def __generate_sobel_shingles(self, folder, width, height):
        scale = 1
        delta = 0
        ddepth = cv.CV_16S
        path, dirs, files = next(os.walk(folder))
        file_count = len(files)
        file_count = file_count
        imageArray = []
        counter = 0
        for i in range(file_count):
            imageArray.append([])
       
        for filename in files:
                exactPath = os.path.join(folder,filename)
                img = cv.imread(exactPath)
                if img is not None:
                    src = cv.imread(exactPath, cv.IMREAD_COLOR)
                    src = cv.resize(src,(width, height))
                    src = cv.GaussianBlur(src, (3, 3), 0)
                    gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
                    grad_x = cv.Sobel(gray, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)
                    # Gradient-Y
                    grad_y = cv.Sobel(gray, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)
                    abs_grad_x = cv.convertScaleAbs(grad_x)
                    abs_grad_y = cv.convertScaleAbs(grad_y)
                    grad = cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
                    for i in range (height):
                        for m in range(width):
                            if(grad[i][m] < 120):
                                grad[i][m] = 0
                                imageArray[counter].append(0)
                            else:
                                grad[i][m] = 255
                                imageArray[counter].append(1)
                    counter = counter + 1
        return np.transpose(imageArray)
    
    ######################################### CNN #########################################
    
    # Gets all images in given folder with given dimensions
    def __generate_cnn_shingles(self, folder, width, height, layer):
        base_model = vgg16.VGG16(weights='imagenet', include_top=True)
        model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer).output)
        all_files = os.listdir(folder)
        all_features = []
        for file in all_files:
            img = image.load_img(os.path.join(folder,file), target_size=(width, height))
            feature = self.__extract_feature(img, model)
            all_features.append(feature)
        return np.transpose(all_features)
    
    # Inputs image to the VGG16 model and gets image features
    def __extract_feature(self,img, model):
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = vgg16.preprocess_input(img_data)
        img_data = np.asarray(img_data)
        # Predict
        vgg16_feature = model.predict(img_data)
        return self.__convert_to_binary(vgg16_feature[0])
    
    # Converts multi-dimensional matrix to binary 1D-matrix
    @staticmethod
    def __convert_to_binary(matrix):
        matrix = matrix.flatten()
        #matrix[matrix != 0] = 1
        #matrix = matrix.astype(int)
        return matrix 

In [3]:
'''
s = Shingling()
a = [4,10,15]
s = Shingling()
_, dirs, _ = next(os.walk("../data"))

for i in range(len(dirs)):
    shingle = s.generate_shingles(os.path.join("../data", dirs[i]), 224, 224, "CNN", layer="predictions")
    with open(dirs[i] + '_cnn_2.pickle', 'wb') as handle:
        pickle.dump(shingle, handle, protocol=pickle.HIGHEST_PROTOCOL)
'''

'\ns = Shingling()\na = [4,10,15]\ns = Shingling()\n_, dirs, _ = next(os.walk("../data"))\n\nfor i in range(len(dirs)):\n    shingle = s.generate_shingles(os.path.join("../data", dirs[i]), 224, 224, "CNN", layer="predictions")\n    with open(dirs[i] + \'_cnn_2.pickle\', \'wb\') as handle:\n        pickle.dump(shingle, handle, protocol=pickle.HIGHEST_PROTOCOL)\n'

In [4]:
%%html
<img src="../resources/layers.jpeg", width=600, height=250>